1. Ensure that we are trading every hour
2. Get live data (3 Data points to account for the data preparation)
3. Append it to csv, ensuring the limit of data points in the csv file (the limit == the train_size for the optimization process)
4. Load the data from the csv file
5. optimize if time to optimize
6. rebalance if time to rebalance
7. Run the strategy on the dataset
8. on the last time index (the last candle)

In [ ]:
import time
from datetime import datetime, timedelta

def run_strategy():
    while True:
        # Get current time and calculate next hour
        now = datetime.now()
        next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
        
        # Wait until the next hour
        sleep_duration = (next_hour - now).total_seconds()
        print(f"Sleeping for {sleep_duration} seconds...")
        time.sleep(sleep_duration)
        
        # Execute your strategy
        print("Running strategy at", datetime.now())
        # Add your trading strategy logic here


In [1]:
pip install apscheduler

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler

def run_strategy():
    print("Running strategy at", datetime.now())
    # Add your trading strategy logic here

scheduler = BlockingScheduler()
scheduler.add_job(run_strategy, 'cron', minute=0)  # Run at the start of every hour
scheduler.start()


In [ ]:
import ccxt
import pandas as pd
import time
import os

# Initialize Kraken Pro Exchange
exchange = ccxt.kraken({
    'apiKey': 'your_api_key',
    'secret': 'your_api_secret'
})

# Fetch the latest OHLCV data point
def fetch_latest_data(symbol, timeframe):
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=1)
        latest_data = ohlcv[-1]
        return {
            'timestamp': pd.to_datetime(latest_data[0], unit='ms'),
            'open': latest_data[1],
            'high': latest_data[2],
            'low': latest_data[3],
            'close': latest_data[4],
            'volume': latest_data[5]
        }
    except Exception as e:
        print(f"Error fetching latest data: {e}")
        return None

# Append new data to CSV and maintain max length
def append_to_csv_with_limit(data, filename, max_rows=3000):
    file_exists = os.path.isfile(filename)
    df = pd.DataFrame([data])
    
    if file_exists:
        existing_df = pd.read_csv(filename)
        combined_df = pd.concat([existing_df, df], ignore_index=True)
        if len(combined_df) > max_rows:
            combined_df = combined_df.iloc[-max_rows:]  # Keep only the last max_rows rows
        combined_df.to_csv(filename, index=False)
    else:
        df.to_csv(filename, mode='w', header=True, index=False)

# Load the dataset from the CSV file
def load_data_from_csv(filename):
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        return pd.DataFrame()

# Calculate moving averages on the dataset
def calculate_moving_averages(df, fast_period, slow_period):
    df['fast_ma'] = df['close'].rolling(window=fast_period).mean()
    df['slow_ma'] = df['close'].rolling(window=slow_period).mean()
    return df

# Check crossover signals
def check_crossover(df):
    if len(df) < 2:
        return None
    if df['fast_ma'].iloc[-2] < df['slow_ma'].iloc[-2] and df['fast_ma'].iloc[-1] > df['slow_ma'].iloc[-1]:
        return 'buy'
    elif df['fast_ma'].iloc[-2] > df['slow_ma'].iloc[-2] and df['fast_ma'].iloc[-1] < df['slow_ma'].iloc[-1]:
        return 'sell'
    else:
        return None

# Main function to run the strategy
def run_strategy(symbol, timeframe, fast_period, slow_period, filename, max_rows):
    print(f"Starting Moving Average Crossover Strategy for {symbol} on {timeframe} timeframe.")
    while True:
        latest_data = fetch_latest_data(symbol, timeframe)
        if latest_data:
            append_to_csv_with_limit(latest_data, filename, max_rows)
            dataset = load_data_from_csv(filename)
            dataset = calculate_moving_averages(dataset, fast_period, slow_period)
            signal = check_crossover(dataset)
            if signal == 'buy':
                print(f"Buy signal detected at {latest_data['timestamp']}")
            elif signal == 'sell':
                print(f"Sell signal detected at {latest_data['timestamp']}")
            else:
                print(f"No signal at {latest_data['timestamp']}")
        time.sleep(60)  # Wait for the next candle

# Run the strategy

symbol = 'BTC/USD'  # Trading pair
timeframe = '1m'  # Timeframe
fast_period = 5  # Fast moving average period
slow_period = 20  # Slow moving average period
csv_filename = 'market_data.csv'  # File to store historical data
max_data_points = 3000  # Maximum number of rows in the CSV

run_strategy(symbol, timeframe, fast_period, slow_period, csv_filename, max_data_points)